---
### 1. Import Dataset and Libraries

In [575]:
#Import Libraries
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt  
import seaborn as sns 
import datetime
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

pd.options.display.max_rows = 75
%matplotlib notebook

from sklearn.preprocessing import PolynomialFeatures

In [576]:
#Import Data
df = pd.read_csv('HRDataset_v14.csv')
pd.set_option('display.max_columns', None)
df

,Employee_Name,EmpID,MarriedID,MaritalStatusID,GenderID,EmpStatusID,DeptID,PerfScoreID,FromDiversityJobFairID,Salary,Termd,PositionID,Position,State,Zip,DOB,Sex,MaritalDesc,CitizenDesc,HispanicLatino,RaceDesc,DateofHire,DateofTermination,TermReason,EmploymentStatus,Department,ManagerName,ManagerID,RecruitmentSource,PerformanceScore,EngagementSurvey,EmpSatisfaction,SpecialProjectsCount,LastPerformanceReview_Date,DaysLateLast30,Absences
0,"Adinolfi, Wilson K",10026,0,0,1,1,5,4,0,62506,0,19,Production Technician I,MA,1960,7/10/1983,M,Single,US Citizen,No,White,7/5/2011,NaN,N/A-StillEmployed,Active,Production,Michael Albert,22.0,LinkedIn,Exceeds,4.60,5,0,1/17/2019,0,1
1,"Ait Sidi, Karthikeyan",10084,1,1,1,5,3,3,0,104437,1,27,Sr. DBA,MA,2148,5/5/1975,M,Married,US Citizen,No,White,3/30/2015,6/16/2016,career change,Voluntarily Terminated,IT/IS,Simon Roup,4.0,Indeed,Fully Meets,4.96,3,6,2/24/2016,0,17
2,"Akinkuolie, Sarah",10196,1,1,0,5,5,3,0,64955,1,20,Production Technician II,MA,1810,9/19/1988,F,Married,US Citizen,No,White,7/5/2011,9/24/2012,hours,Voluntarily Terminated,Production,Kissy Sullivan,20.0,LinkedIn,Fully Meets,3.02,3,0,5/15/2012,0,3
3,"Alagbe,Trina",10088,1,1,0,1,5,3,0,64991,0,19,Production Technician I,MA,1886,9/27/1988,F,Married,US Citizen,No,White,1/7/2008,NaN,N/A-StillEmployed,Active,Production,Elijiah Gray,16.0,Indeed,Fully Meets,4.84,5,0,1/3/2019,0,15
4,"Anderson, Carol",10069,0,2,0,5,5,3,0,50825,1,19,Production Technician I,MA,2169,9/8/1989,F,Divorced,US Citizen,No,White,7/11/2011,9/6/2016,return to school,Voluntarily Terminated,Production,Webster Butler,39.0,Google Search,Fully Meets,5.00,4,0,2/1/2016,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306,"Woodson, Jason",10135,0,0,1,1,5,3,0,65893,0,20,Production Technician II,MA,1810,5/11/1985,M,Single,US Citizen,No,White,7/7/2014,NaN,N/A-StillEmployed,Active,Production,Kissy Sullivan,20.0,LinkedIn,Fully Meets,4.07,4,0,2/28/2019,0,13
307,"Ybarra, Catherine",10301,0,0,0,5,5,1,0,48513,1,19,Production Technician I,MA,2458,5/4/1982,F,Single,US Citizen,No,Asian,9/2/2008,9/29/2015,Another position,Voluntarily Terminated,Production,Brannon Miller,12.0,Google Search,PIP,3.20,2,0,9/2/2015,5,4
308,"Zamora, Jennifer",10010,0,0,0,1,3,4,0,220450,0,6,CIO,MA,2067,8/30/1979,F,Single,US Citizen,No,White,4/10/2010,NaN,N/A-StillEmployed,Active,IT/IS,Janet King,2.0,Employee Referral,Exceeds,4.60,5,6,2/21/2019,0,16
309,"Zhou, Julia",10043,0,0,0,1,3,3,0,89292,0,9,Data Analyst,MA,2148,2/24/1979,F,Single,US Citizen,No,White,3/30/2015,NaN,N/A-StillEmployed,Active,IT/IS,Simon Roup,4.0,Employee Referral,Fully Meets,5.00,3,5,2/1/2019,0,11


---
### 2. Data Sanitzation and Enrichment

In [577]:
#Only keeping relevant columns

df = df[['EmpID','MarriedID', 'GenderID', 'Salary', 'DOB', 'CitizenDesc', 'RaceDesc', 'DateofHire', 'DateofTermination', 'TermReason', 'Sex', 'MaritalDesc', 'EmploymentStatus', 'PerformanceScore', 'EngagementSurvey', 'EmpSatisfaction', 'Absences']]
df

,EmpID,MarriedID,GenderID,Salary,DOB,CitizenDesc,RaceDesc,DateofHire,DateofTermination,TermReason,Sex,MaritalDesc,EmploymentStatus,PerformanceScore,EngagementSurvey,EmpSatisfaction,Absences
0,10026,0,1,62506,7/10/1983,US Citizen,White,7/5/2011,NaN,N/A-StillEmployed,M,Single,Active,Exceeds,4.60,5,1
1,10084,1,1,104437,5/5/1975,US Citizen,White,3/30/2015,6/16/2016,career change,M,Married,Voluntarily Terminated,Fully Meets,4.96,3,17
2,10196,1,0,64955,9/19/1988,US Citizen,White,7/5/2011,9/24/2012,hours,F,Married,Voluntarily Terminated,Fully Meets,3.02,3,3
3,10088,1,0,64991,9/27/1988,US Citizen,White,1/7/2008,NaN,N/A-StillEmployed,F,Married,Active,Fully Meets,4.84,5,15
4,10069,0,0,50825,9/8/1989,US Citizen,White,7/11/2011,9/6/2016,return to school,F,Divorced,Voluntarily Terminated,Fully Meets,5.00,4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306,10135,0,1,65893,5/11/1985,US Citizen,White,7/7/2014,NaN,N/A-StillEmployed,M,Single,Active,Fully Meets,4.07,4,13
307,10301,0,0,48513,5/4/1982,US Citizen,Asian,9/2/2008,9/29/2015,Another position,F,Single,Voluntarily Terminated,PIP,3.20,2,4
308,10010,0,0,220450,8/30/1979,US Citizen,White,4/10/2010,NaN,N/A-StillEmployed,F,Single,Active,Exceeds,4.60,5,16
309,10043,0,0,89292,2/24/1979,US Citizen,White,3/30/2015,NaN,N/A-StillEmployed,F,Single,Active,Fully Meets,5.00,3,11


We would like to only analyze employees that still work at the company against employees that quit from the company. Hence, let's remove any employees that were fired from the dataset.

In [578]:
# Remove employees that were fired from the dataset
df = df[df['EmploymentStatus'] != 'Terminated for Cause']
df['EmploymentStatus'].unique()

array(['Active', 'Voluntarily Terminated'], dtype=object)

Lets replace the column titled "DOB" with a column that explicitly states the date of the employee as of present day. 

In [579]:
# Convert DOB column to datetime
df['DOB'] = pd.to_datetime(df['DOB'])

# Calculate the current age of each employee
now = pd.Timestamp.now()
df['Age'] = ((now - df['DOB']).dt.days / 365.25).astype(int)

# Drop the DOB column from the df dataset
df = df.drop('DOB', axis=1)

df.head(5)

/tmp/ipykernel_7187/2330545023.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DOB'] = pd.to_datetime(df['DOB'])
/tmp/ipykernel_7187/2330545023.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Age'] = ((now - df['DOB']).dt.days / 365.25).astype(int)


,EmpID,MarriedID,GenderID,Salary,CitizenDesc,RaceDesc,DateofHire,DateofTermination,TermReason,Sex,MaritalDesc,EmploymentStatus,PerformanceScore,EngagementSurvey,EmpSatisfaction,Absences,Age
0,10026,0,1,62506,US Citizen,White,7/5/2011,NaN,N/A-StillEmployed,M,Single,Active,Exceeds,4.60,5,1,39
1,10084,1,1,104437,US Citizen,White,3/30/2015,6/16/2016,career change,M,Married,Voluntarily Terminated,Fully Meets,4.96,3,17,47
2,10196,1,0,64955,US Citizen,White,7/5/2011,9/24/2012,hours,F,Married,Voluntarily Terminated,Fully Meets,3.02,3,3,34
3,10088,1,0,64991,US Citizen,White,1/7/2008,NaN,N/A-StillEmployed,F,Married,Active,Fully Meets,4.84,5,15,34
4,10069,0,0,50825,US Citizen,White,7/11/2011,9/6/2016,return to school,F,Divorced,Voluntarily Terminated,Fully Meets,5.00,4,2,33


Let's simplify the MaritalDesc column.

In [580]:
# Replace values in MaritalDesc column
df["MaritalDesc"].replace({"Divorced": "Single", "Widowed": "Single", "Separated": "Married"}, inplace=True)
df['MaritalDesc'].unique()

array(['Single', 'Married'], dtype=object)

Let's calculate the duration of the employees' time at the company.

For active employees, we will calculate their time spent at the company using "DateofHire" to present day. For terminated employees, we will caluclate their time spent at the company using "DateofHire" and "DateofTermination."

In [581]:
# convert DateofHire and DateofTermination columns to datetime
df['DateofHire'] = pd.to_datetime(df['DateofHire'])
df['DateofTermination'] = pd.to_datetime(df['DateofTermination'])

# create a new column called 'LengthofEmployment'
df['LengthofEmployment'] = ""

today = datetime.datetime.now()

# calculate length of employment
def calculate_length_of_employment(row):
    if row['EmploymentStatus'] == 'Active' and pd.isnull(row['DateofTermination']):
        length_of_employment = (today - row['DateofHire']).days / 365
    elif row['EmploymentStatus'] == 'Voluntarily Terminated':
        length_of_employment = (row['DateofTermination'] - row['DateofHire']).days / 365
    else:
        length_of_employment = None
    return length_of_employment

df['LengthofEmployment'] = df.apply(calculate_length_of_employment, axis=1)

#get rid of the columns 
df = df.drop(columns=['DateofHire', 'DateofTermination'])
df

,EmpID,MarriedID,GenderID,Salary,CitizenDesc,RaceDesc,TermReason,Sex,MaritalDesc,EmploymentStatus,PerformanceScore,EngagementSurvey,EmpSatisfaction,Absences,Age,LengthofEmployment
0,10026,0,1,62506,US Citizen,White,N/A-StillEmployed,M,Single,Active,Exceeds,4.60,5,1,39,11.810959
1,10084,1,1,104437,US Citizen,White,career change,M,Married,Voluntarily Terminated,Fully Meets,4.96,3,17,47,1.216438
2,10196,1,0,64955,US Citizen,White,hours,F,Married,Voluntarily Terminated,Fully Meets,3.02,3,3,34,1.224658
3,10088,1,0,64991,US Citizen,White,N/A-StillEmployed,F,Married,Active,Fully Meets,4.84,5,15,34,15.304110
4,10069,0,0,50825,US Citizen,White,return to school,F,Single,Voluntarily Terminated,Fully Meets,5.00,4,2,33,5.161644
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306,10135,0,1,65893,US Citizen,White,N/A-StillEmployed,M,Single,Active,Fully Meets,4.07,4,13,37,8.802740
307,10301,0,0,48513,US Citizen,Asian,Another position,F,Single,Voluntarily Terminated,PIP,3.20,2,4,40,7.076712
308,10010,0,0,220450,US Citizen,White,N/A-StillEmployed,F,Single,Active,Exceeds,4.60,5,16,43,13.046575
309,10043,0,0,89292,US Citizen,White,N/A-StillEmployed,F,Single,Active,Fully Meets,5.00,3,11,44,8.073973


Let's change the column CitizenDesc to indices that represent the unique values. 

In [582]:
df_int = df

df = df.drop(['MarriedID', 'GenderID'], axis=1)

In [583]:
#Assign citizen description an integer value
citizen_to_int = {'US Citizen':2,'Eligible NonCitizen':1,'Non-Citizen':0}
df_int['Citizen_int'] = df_int['CitizenDesc'].map(citizen_to_int)
df_int = df_int.drop('CitizenDesc',axis=1)

#Assign performance score an integer value
score_to_int = {'PIP': 1, 'Needs Improvement': 2, 'Fully Meets':3,'Exceeds':4}
df_int['PerformanceScore_Int'] = df_int['PerformanceScore'].map(score_to_int)
df_int = df_int.drop('PerformanceScore',axis=1)

#Assign races an integer value
race_to_int = {'American Indian or Alaska Native': 1, 'Asian':2, 'Black or African American':3, 'Hispanic':4, 'Two or more races':5, 'White':6}
df_int['RaceDesc_Int'] = df_int['RaceDesc'].map(race_to_int)
df_int = df_int.drop('RaceDesc',axis=1)

#Assign employment status an integer value
EmpStat_int = {'Active':1,'Voluntarily Terminated':0}
df_int['EmpStatus_Int'] = df_int['EmploymentStatus'].map(EmpStat_int)
df_int = df_int.drop('EmploymentStatus', axis=1)
df_int


,EmpID,MarriedID,GenderID,Salary,TermReason,Sex,MaritalDesc,EngagementSurvey,EmpSatisfaction,Absences,Age,LengthofEmployment,Citizen_int,PerformanceScore_Int,RaceDesc_Int,EmpStatus_Int
0,10026,0,1,62506,N/A-StillEmployed,M,Single,4.60,5,1,39,11.810959,2,4,6,1
1,10084,1,1,104437,career change,M,Married,4.96,3,17,47,1.216438,2,3,6,0
2,10196,1,0,64955,hours,F,Married,3.02,3,3,34,1.224658,2,3,6,0
3,10088,1,0,64991,N/A-StillEmployed,F,Married,4.84,5,15,34,15.304110,2,3,6,1
4,10069,0,0,50825,return to school,F,Single,5.00,4,2,33,5.161644,2,3,6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306,10135,0,1,65893,N/A-StillEmployed,M,Single,4.07,4,13,37,8.802740,2,3,6,1
307,10301,0,0,48513,Another position,F,Single,3.20,2,4,40,7.076712,2,1,2,0
308,10010,0,0,220450,N/A-StillEmployed,F,Single,4.60,5,16,43,13.046575,2,4,6,1
309,10043,0,0,89292,N/A-StillEmployed,F,Single,5.00,3,11,44,8.073973,2,3,6,1


Let's split this dataset into two so that we can compare the data. One will be for employees with an employment status of "Active," and the other is for "Voluntarily Terminated."

In [584]:
# Create a subdataset for active employees
active_df = df.loc[df['EmploymentStatus'] == 'Active']
active_df = active_df.drop(['EmploymentStatus', 'TermReason'], axis=1)
active_df_int = df_int.loc[df_int['EmpStatus_Int'] == 1]

# Create a subdataset for employees who quit
terminated_df = df.loc[df['EmploymentStatus'] == 'Voluntarily Terminated']
terminated_df = terminated_df.drop('EmploymentStatus', axis=1)
terminated_df_int = df_int.loc[df_int['EmpStatus_Int'] == 0]

---
### 3. Data Exploration

Let's create a visual representation of the correlations between all the factors that interest us. 
1. Heatmap for active
2. Heatmap for voluntarily terminated employees

In [585]:
# create correlation matrix for both active and terminated employees
active_corr_matrix = active_df_int[['Salary', 'Citizen_int', 'PerformanceScore_Int', 'EngagementSurvey', 'EmpSatisfaction', 'Absences', 'Age', 'LengthofEmployment']].corr()

# create heatmap plot
fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(active_corr_matrix, annot=True, cmap='coolwarm', ax=ax)
fig.savefig('ACTIVEcorrelation_matrix.png')

<IPython.core.display.Javascript object>

In [586]:
# create correlation matrix for both active and terminated employees
term_corr_matrix = terminated_df_int[['Salary', 'Citizen_int', 'PerformanceScore_Int', 'EngagementSurvey', 'EmpSatisfaction', 'Absences', 'Age', 'LengthofEmployment']].corr()

# create heatmap plot
fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(term_corr_matrix, annot=True, cmap='coolwarm', ax=ax)
fig.savefig('TERMINATEDcorrelation_matrix.png')

<IPython.core.display.Javascript object>

Now let's understand the demographics of our active and terminated employees.
1. Bar graphs for active employees
2. Bar graphs for voluntarily terminated employees

In [587]:
# create a list of columns to create bar graphs for
columns = ['RaceDesc', 'Sex', 'MaritalDesc', 'PerformanceScore', 'EmpSatisfaction']

# create a loop to create bar graphs for each column
fig, axes = plt.subplots(nrows=1, ncols=len(columns), figsize=(55, 5))
for i, column in enumerate(columns):
    # group the data by the column and count the number of occurrences
    data_counts = active_df.groupby(column).size()

    # create a bar graph
    ax = axes[i]
    ax.bar(data_counts.index, data_counts.values)
    ax.set_title(column)
    ax.set_xlabel(column)
    ax.set_ylabel('Count')
    fig.savefig(f'ACTIVE {column}_bar_plot.png')
plt.show()

<IPython.core.display.Javascript object>

In [588]:
# create a list of columns to create bar graphs for
columns = ['RaceDesc', 'Sex', 'MaritalDesc', 'PerformanceScore', 'EmpSatisfaction']

# create a loop to create bar graphs for each column
fig, axes = plt.subplots(nrows=1, ncols=len(columns), figsize=(55, 5))
for i, column in enumerate(columns):
    # group the data by the column and count the number of occurrences
    data_counts = terminated_df.groupby(column).size()

    # create a bar graph
    ax = axes[i]
    ax.bar(data_counts.index, data_counts.values)
    ax.set_title(column)
    ax.set_xlabel(column)
    ax.set_ylabel('Count')
    fig.savefig(f'TERMINATED {column}_bar_plot.png')
plt.show()


<IPython.core.display.Javascript object>

The following data is nondsicrete and can be evaluated using scatter plots. In case you would like to see the relationship of Salary, Absences, Age, Length of Employment, and the Engagement Survey responses, view the image below. 

2. Scatter Plots for nondiscrete factors

In [589]:
sns.pairplot(data=df[['Salary', 'Absences','Age', 'LengthofEmployment', 'EngagementSurvey']], diag_kind='kde')
plt.savefig('correlation_charts.png')

<IPython.core.display.Javascript object>

---
### 4. Linear Regression & Model Evaluation

In [590]:
##Analyzing above charts for quick future reference
#Time at company vs race: The majority of the people working for the company are White (1) or Black (2). White people tend to stay at the company longer than the other races, which could lead to a computer picking up a racial bias
#Absences vs Race: There are only two people of mixed race working at the company and both of them have a large number of absences (could lead to racial bias). The absences of the white employees are very evenly distributed. 

In [591]:
##Time to make a classification model to predict what employees are gonna quit
#Split the dataset into training and test sets
df_test = df_int
df_test = df_test.drop('TermReason', axis=1)
X = df_test.drop('EmpStatus_Int',axis=1)
Y = df_test['EmpStatus_Int']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2,random_state=42)

df_test

,EmpID,MarriedID,GenderID,Salary,Sex,MaritalDesc,EngagementSurvey,EmpSatisfaction,Absences,Age,LengthofEmployment,Citizen_int,PerformanceScore_Int,RaceDesc_Int,EmpStatus_Int
0,10026,0,1,62506,M,Single,4.60,5,1,39,11.810959,2,4,6,1
1,10084,1,1,104437,M,Married,4.96,3,17,47,1.216438,2,3,6,0
2,10196,1,0,64955,F,Married,3.02,3,3,34,1.224658,2,3,6,0
3,10088,1,0,64991,F,Married,4.84,5,15,34,15.304110,2,3,6,1
4,10069,0,0,50825,F,Single,5.00,4,2,33,5.161644,2,3,6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306,10135,0,1,65893,M,Single,4.07,4,13,37,8.802740,2,3,6,1
307,10301,0,0,48513,F,Single,3.20,2,4,40,7.076712,2,1,2,0
308,10010,0,0,220450,F,Single,4.60,5,16,43,13.046575,2,4,6,1
309,10043,0,0,89292,F,Single,5.00,3,11,44,8.073973,2,3,6,1


In [592]:
print(df_test.dtypes)

EmpID                     int64
MarriedID                 int64
GenderID                  int64
Salary                    int64
Sex                      object
MaritalDesc              object
EngagementSurvey        float64
EmpSatisfaction           int64
Absences                  int64
Age                       int64
LengthofEmployment      float64
Citizen_int               int64
PerformanceScore_Int      int64
RaceDesc_Int              int64
EmpStatus_Int             int64
dtype: object


In [593]:
#Change columns to same data type
df_test['EngagementSurvey'] = df_test['EngagementSurvey'].astype('int64')
df_test['LengthofEmployment'] = df_test['LengthofEmployment'].astype('int64')
print(df_test.dtypes)

EmpID                    int64
MarriedID                int64
GenderID                 int64
Salary                   int64
Sex                     object
MaritalDesc             object
EngagementSurvey         int64
EmpSatisfaction          int64
Absences                 int64
Age                      int64
LengthofEmployment       int64
Citizen_int              int64
PerformanceScore_Int     int64
RaceDesc_Int             int64
EmpStatus_Int            int64
dtype: object


In [594]:
# Create KNN classifier with k=3
knn = KNeighborsClassifier(n_neighbors=4)

# Fit the classifier to the training data
knn.fit(X_train, Y_train)

# Make predictions on the testing data
y_pred = knn.predict(X_test)

#print('Mean Absolute Error:', metrics.mean_absolute_error(Y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(Y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(Y_test, y_pred)))
print("Score for Number of Neighbors = 3: {}".format(knn.score(X_test,Y_test)))
conf_mat = confusion_matrix(Y_test,y_pred)
print('confusion matrix: \n', conf_mat)
print("Accuracy:", accuracy_score(Y_test,y_pred))


ValueError: could not convert string to float: 'F'